In [119]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [120]:
np.set_printoptions(threshold=np.inf)

In [148]:
wm_image = cv2.imread('Images\WM.png', cv2.IMREAD_GRAYSCALE)
input_image = cv2.imread('Images\InputImage.png')

In [122]:
_, wm_binary = cv2.threshold(wm_image,165, 255, cv2.THRESH_BINARY)

In [ ]:
def update_wm(image, h_new, w_new):
    """
    Make watermark as large as container
    """
    h, w = image.shape
    while True:
        image = np.concatenate((image, image), axis=1)
        if image.shape[1]>w_new:
            break
    while True:
        image = np.concatenate((image, image), axis=0)
        if image.shape[0]>h_new:
            break
    image = image[:h_new, :w_new]
    return image

In [ ]:
def insert_wm(container, wm, bit):
    """
    Inserting watermark into image
    """
    h_c, w_c, _ = container.shape
    h_wm,w_wm = wm.shape
    # if container bigger
    if h_wm < h_c or w_wm < w_c:
        wm = update_wm(wm, h_c, w_c)
    # if container smaller  
    if h_wm > h_c or w_wm > w_c:
        wm = wm[:h_c, :w_c]

    # making image 0 and 1
    wm = wm & 1
    # making three streams
    r, g, b = cv2.split(container)

    # Bit demonstration of int8 values
    b_unpacked = np.unpackbits(b[:,:,np.newaxis], axis=2)
    # Changing bits
    b_unpacked[:,:, -bit] = wm[:, :]
    # Change bit demonstration to int8
    b_packed = np.packbits(b_unpacked, axis=2)
    b_new = b_packed.reshape(b_packed.shape[0],b_packed.shape[1])
    # Making new image
    modified_image = cv2.merge((r, g, b_new))
    return modified_image

In [ ]:
def extrect_wm(container, bit):
    """
    Extrecting watermark from image
    """
    _, _, b_mod = cv2.split(container)
    b_mod_unpacked = np.unpackbits(b_mod[:,:,np.newaxis], axis=2)
    wm_from_im = b_mod_unpacked[:,:, -bit]
    return wm_from_im

In [ ]:
image_with_watermark = insert_wm(input_image, wm_binary, 1)
plt.imshow(image_with_watermark)

In [ ]:
watermark_from_image = extrect_wm(image_with_watermark, bit=1)
plt.imshow(watermark_from_image, cmap='gray')

In [189]:
def text_to_binary(text):
    """
    Преобразует текст в бинарный формат ASCII
    """
    binary_text = ''.join(format(ord(char), '08b') for char in text)
    return binary_text

In [186]:
r, g, b = cv2.split(input_image)
b_unpacked = np.unpackbits(b[:,:,np.newaxis], axis=2)

for i in range(b_unpacked.shape[0]):
    for j in range(b_unpacked.shape[1]):
        if text == '':
            break
        b_unpacked[i, j, -1] = text[0]
        text = text[1:]


In [201]:
def insert_text(container, text, bit):
    binary_text = text_to_binary(text)
    h_c, w_c, _ = container.shape
    length_text = len(binary_text)
    # текст слишком большой
    if h_c*w_c < length_text:
        return
    _, _, b = cv2.split(input_image)
    b_unpacked = np.unpackbits(b[:,:,np.newaxis], axis=2)

    for i in range(b_unpacked.shape[0]):
        for j in range(b_unpacked.shape[1]):
            if binary_text == '':
                break
            b_unpacked[i, j, -bit] = binary_text[0]
            binary_text = binary_text[1:]
    # Change bit demonstration to int8
    b_packed = np.packbits(b_unpacked, axis=2)
    b_new = b_packed.reshape(b_packed.shape[0],b_packed.shape[1])
    # Making new image
    modified_image = cv2.merge((r, g, b_new))
    return modified_image, length_text


In [196]:
_, _, b_mod = cv2.split(input_image)
b_mod_unpacked = np.unpackbits(b_mod[:,:,np.newaxis], axis=2)
text_from_im = b_mod_unpacked[:,:, -1].flatten()
text_from_im[:8]

array([1, 1, 1, 0, 0, 1, 1, 0], dtype=uint8)

In [198]:
def binary_to_text(binary_text):
    """
    Преобразует бинарную строку обратно в текст
    """
    # Разбиваем бинарную строку на восемьбитовые части
    chunks = [binary_text[i:i+8] for i in range(0, len(binary_text), 8)]
    
    # Преобразуем каждую восемьбитовую часть в число и затем в символ ASCII
    text = ''.join([chr(int(chunk, 2)) for chunk in chunks])
    
    return text


In [199]:
def extrect_text(container, length_text):
    _, _, b_mod = cv2.split(container)
    b_mod_unpacked = np.unpackbits(b_mod[:,:,np.newaxis], axis=2)
    text_from_im = b_mod_unpacked[:,:, -1].flatten()
    encrypted_text = text_from_im[:length_text]
    encrypted_text_str = ''.join(map(str, encrypted_text))
    return binary_to_text(encrypted_text_str)


In [202]:
im, l = insert_text(input_image, 'sex', 1)

In [203]:
extrect_text(im, l)

'sex'